# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [97]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords', 'averaged_perceptron_tagger'])

# import statements

import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tag import pos_tag

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sqlalchemy import create_engine


from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report, recall_score, precision_score
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /Users/mo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/mo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [98]:
# load data from database
engine = create_engine('sqlite:///df_clean.db')
df = pd.read_sql_table("messages", engine)
df = df[df.related != 2]
X = df["message"]
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [99]:
def tokenize(text):
    
    url_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url,'urlplaceholder')

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    #clean_tokens = [x for x in clean_tokens if x not in punc]    
    return clean_tokens

In [100]:
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', '-', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over'] 

Looking for someone but no name
['looking', 'for', 'someone', 'but', 'no', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80-90', 'destroyed', '.', 'only', 'hospital', 'st.', 'croix', 'functioning', '.', 'needs', 'supply', 'desperately', '.'] 

says: west side of Haiti, rest of the country today and tonight
['say', ':', 'west', 'side', 'of', 'haiti', ',', 'rest', 'of', 'the', 'country', 'today', 'and', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [101]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42)))
])

pipeline2 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf2', MultiOutputClassifier(AdaBoostClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Pipeline 1

In [27]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fbab7e23ca0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)))])

In [28]:
y_pred = pipeline.predict(X_test)

# Training accuracy
print("The training accuracy is: ")
print(accuracy_score(y_train, pipeline.predict(X_train))*100)
print(" ")

The training accuracy is: 
99.77460171097792
 


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [40]:
y_preddb = pd.DataFrame(y_pred)
y_preddb.columns = y_test.columns
y_preddb.index = y_test.index
print(classification_report(y_test, y_preddb, target_names=y_preddb.columns))

                        precision    recall  f1-score   support

               related       0.81      0.97      0.88      4998
               request       0.90      0.45      0.60      1152
                 offer       0.00      0.00      0.00        37
           aid_related       0.80      0.61      0.69      2755
          medical_help       0.60      0.02      0.04       516
      medical_products       0.61      0.04      0.08       317
     search_and_rescue       0.64      0.04      0.07       187
              security       0.50      0.01      0.02       118
              military       0.64      0.04      0.07       228
           child_alone       0.00      0.00      0.00         0
                 water       0.94      0.29      0.44       401
                  food       0.89      0.40      0.55       755
               shelter       0.90      0.28      0.43       603
              clothing       0.64      0.08      0.14        90
                 money       0.71      

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_cla

### 6. Improve your model
Use grid search to find better parameters. 

In [46]:
pipeline = Pipeline([
    ("vect", CountVectorizer(tokenizer=tokenize)),
    ("tfidf", TfidfTransformer()),
    ("clf", MultiOutputClassifier(RandomForestClassifier()))
])
    
parameters = {'clf__estimator__n_estimators':[25,50,100],
              "clf__estimator__max_depth": [25, 50, 100],
              "clf__estimator__max_features": ["sqrt", "auto"]
             }

    
cv = GridSearchCV(pipeline, parameters)
cv.fit(X_train, y_train)
pipe = cv

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = pipe.predict(X_test)

# overall accuracy
accuracy = (y_pred == y_test).mean().mean()
print('Accuracy {0:.2f}% \n'.format(accuracy*100))

# If some labels are not predicted at least once, Y_pred will have different 
# columns than Y_test, which will cause an error in the classification_report()
# So make sure Y_pred has the same labels as Y_test.

y_pred = pd.DataFrame(y_pred);
y_pred.columns = y_test.columns;
y_pred.index = y_test.index;

In [52]:
print(classification_report(y_test, y_pred, target_names=y_pred.columns))

                        precision    recall  f1-score   support

               related       0.80      0.98      0.88      4998
               request       0.91      0.40      0.56      1152
                 offer       0.00      0.00      0.00        37
           aid_related       0.81      0.59      0.69      2755
          medical_help       0.62      0.03      0.05       516
      medical_products       0.59      0.03      0.06       317
     search_and_rescue       0.67      0.01      0.02       187
              security       0.00      0.00      0.00       118
              military       1.00      0.03      0.05       228
           child_alone       0.00      0.00      0.00         0
                 water       0.95      0.15      0.25       401
                  food       0.89      0.30      0.45       755
               shelter       0.90      0.19      0.31       603
              clothing       0.64      0.08      0.14        90
                 money       1.00      

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_cla

In [51]:
print("\nBest Parameters:", pipe.best_params_)


Best Parameters: {'clf__estimator__max_depth': 100, 'clf__estimator__max_features': 'sqrt', 'clf__estimator__n_estimators': 100}


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [89]:
c = DecisionTreeClassifier()
c.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

In [90]:
pipeline3 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
])

parameters3 = {
    'clf__estimator__max_features': ['auto', 'sqrt', 'log2'],
    'clf__estimator__max_depth': [1,5,7]
} 

In [91]:
cv2 = GridSearchCV(pipeline3, parameters3).fit(X_train, y_train)
pipe2 = cv2

In [93]:
y_pred = pipe2.predict(X_test)

# overall accuracy
accuracy = (y_pred == y_test).mean().mean()
print('Accuracy {0:.2f}% \n'.format(accuracy*100))

# If some labels are not predicted at least once, Y_pred will have different 
# columns than Y_test, which will cause an error in the classification_report()
# So make sure Y_pred has the same labels as Y_test.

y_pred = pd.DataFrame(y_pred);
y_pred.columns = y_test.columns;
y_pred.index = y_test.index;

Accuracy 92.58% 



In [94]:
print(classification_report(y_test, y_pred, target_names=y_pred.columns))

                        precision    recall  f1-score   support

               related       0.77      1.00      0.87      4998
               request       1.00      0.00      0.00      1152
                 offer       0.00      0.00      0.00        37
           aid_related       0.79      0.01      0.01      2755
          medical_help       0.50      0.01      0.02       516
      medical_products       0.10      0.00      0.01       317
     search_and_rescue       0.00      0.00      0.00       187
              security       0.00      0.00      0.00       118
              military       0.00      0.00      0.00       228
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       401
                  food       0.00      0.00      0.00       755
               shelter       0.00      0.00      0.00       603
              clothing       0.00      0.00      0.00        90
                 money       0.00      

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_cla

In [96]:
print("\nBest Parameters:", pipe2.best_params_)


Best Parameters: {'clf__estimator__max_depth': 5, 'clf__estimator__max_features': 'log2'}


### 9. Export your model as a pickle file

In [95]:
import pickle
ML_pipeline = pickle.dumps('classifier.pkl')